In [1]:
import pandas as pd
import numpy as np
from math import isnan

#### Part 1

In [2]:
df = pd.read_csv('attribution_allocation_student_data.csv')
df.head()

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,referral,referral,referral,email,NaN,1
1,True,referral,display,display,email,NaN,1
2,True,email,display,referral,NaN,NaN,1
3,True,referral,referral,email,NaN,NaN,1
4,True,social,referral,NaN,NaN,NaN,1


In [3]:
len(df)

68124

In [4]:
df = df[df['convert_TF']==True]
df.head()

,convert_TF,touch1,touch2,touch3,touch4,touch5,tier
0,True,referral,referral,referral,email,NaN,1
1,True,referral,display,display,email,NaN,1
2,True,email,display,referral,NaN,NaN,1
3,True,referral,referral,email,NaN,NaN,1
4,True,social,referral,NaN,NaN,NaN,1


In [5]:
# How many users converted?
df[df['convert_TF'] == True].shape[0]

13779

In [6]:
channel_spend = pd.read_csv('channel_spend_student_data.csv')
channel_spend

,Unnamed: 0,0
0,tier1,"{'email': 1000.0, 'social': 1000.0000000000001..."
1,tier2,"{'email': 2000.0, 'social': 2000.0, 'display':..."
2,tier3,"{'email': 3000.0, 'social': 3000.0, 'display':..."


In [7]:
dic = eval(channel_spend['0'][0])
dic

{'email': 1000.0,
 'social': 1000.0000000000001,
 'display': 1000.0,
 'paid_search': 999.9999999999999,
 'referral': 1000.0,
 'organic_search': 0.0,
 'direct': 0.0}

In [8]:
dic['social'] = round(dic['social'],1)
dic['paid_search'] = round(dic['paid_search'],1)
dic

{'email': 1000.0,
 'social': 1000.0,
 'display': 1000.0,
 'paid_search': 1000.0,
 'referral': 1000.0,
 'organic_search': 0.0,
 'direct': 0.0}

In [9]:
channel_spend['0'][0] = str(dic)
channel_spend

,Unnamed: 0,0
0,tier1,"{'email': 1000.0, 'social': 1000.0, 'display':..."
1,tier2,"{'email': 2000.0, 'social': 2000.0, 'display':..."
2,tier3,"{'email': 3000.0, 'social': 3000.0, 'display':..."


In [10]:
channel_spend.rename(columns = {"Unnamed: 0": "tier","0":"channel_spend"},inplace = True)
channel_spend

,tier,channel_spend
0,tier1,"{'email': 1000.0, 'social': 1000.0, 'display':..."
1,tier2,"{'email': 2000.0, 'social': 2000.0, 'display':..."
2,tier3,"{'email': 3000.0, 'social': 3000.0, 'display':..."


In [11]:
def last_interaction(df):
    touchs=['touch1','touch2','touch3','touch4','touch5']
    data=pd.DataFrame(columns=['tier','touch'])
    for i in df.index:
        data.loc[i]=[df['tier'][i],df[touchs[4-df.loc[i].isnull().sum()]][i]]
    return data
last_touch_data = last_interaction(df)

In [33]:
data1=last_touch_data[last_touch_data['tier']==1]
data2=last_touch_data[last_touch_data['tier']==2]
data3=last_touch_data[last_touch_data['tier']==3]
row=['email','organic_search','referral','social','paid_search','display','direct']
col=['tier1','tier2','tier3']
last=pd.DataFrame(columns=col,index=row)
last['tier1']=data1['touch'].value_counts()
last['tier2']=data2['touch'].value_counts()
last['tier3']=data3['touch'].value_counts()
last['sum'] = last['tier1'] + last['tier2'] + last['tier3']
# last['avg_cac'] = 6000 / last ['sum']
# 对last按照数量排序
last=last.sort_values(by='tier1',ascending=False)
last

,tier1,tier2,tier3,sum,avg_cac
referral,1211,2107,2696,6014,0.997672
social,625,1060,1275,2960,2.027027
display,528,818,946,2292,2.617801
email,281,555,655,1491,4.024145
paid_search,176,347,481,1004,5.976096
organic_search,2,4,5,11,545.454545
direct,1,1,5,7,857.142857


In [13]:
def first_interaction(df):
    data=pd.DataFrame(columns=['tier','touch'])
    data['tier']=df['tier']
    data['touch']=df['touch1']
    return data
first_touch_data = first_interaction(df)
first_touch_data

,tier,touch
0,1,referral
1,1,referral
2,1,email
3,1,referral
4,1,social
...,...,...
60648,3,referral
60649,3,display
68103,3,organic_search
68104,3,organic_search


In [32]:
data1=first_touch_data[first_touch_data['tier']==1]
data2=first_touch_data[first_touch_data['tier']==2]
data3=first_touch_data[first_touch_data['tier']==3]
row=['email','organic_search','referral','social','paid_search','display','direct']
col=['tier1','tier2','tier3']
first=pd.DataFrame(columns=col,index=row)
first['tier1']=data1['touch'].value_counts()
first['tier2']=data2['touch'].value_counts()
first['tier3']=data3['touch'].value_counts()

first['sum'] = first['tier1'] + first['tier2'] + first['tier3']
first['avg_cac'] = 6000 / first ['sum']
# 对first按照数量排序
first=first.sort_values(by='tier1',ascending=False)
first

,tier1,tier2,tier3,sum,avg_cac
referral,1519,2719,3257,7495,0.800534
social,475,823,982,2280,2.631579
display,435,726,870,2031,2.954210
email,264,402,618,1284,4.672897
paid_search,129,211,328,668,8.982036
organic_search,1,7,7,15,400.000000
direct,1,4,1,6,1000.000000


In [15]:
def linear(df):
    dic1={'email':0,'organic_search':0,'referral':0,'social':0,'paid_search':0,'display':0,'direct':0}
    dic2={'email':0,'organic_search':0,'referral':0,'social':0,'paid_search':0,'display':0,'direct':0}
    dic3={'email':0,'organic_search':0,'referral':0,'social':0,'paid_search':0,'display':0,'direct':0}
    touchs=['touch1','touch2','touch3','touch4','touch5']
    for i in df.index:
        if df['tier'][i]==1:
            n=df.loc[i].isnull().sum()
            score=1/(5-n)
            for touch in touchs[:5-n]:
                dic1[df[touch][i]]+=score
        elif df['tier'][i]==2:
            n=df.loc[i].isnull().sum()
            score=1/(5-n)
            for touch in touchs[:5-n]:
                dic2[df[touch][i]]+=score
        else:
            n=df.loc[i].isnull().sum()
            score=1/(5-n)
            for touch in touchs[:5-n]:
                dic3[df[touch][i]]+=score
    return dic1,dic2,dic3
tier1_dic,tier2_dic,tier3_dic = linear(df)

In [31]:
row=['email','organic_search','referral','social','paid_search','display','direct']
col=['tier1','tier2','tier3']
linear_data=pd.DataFrame(columns=col,index=row)
linear_data['tier1']=pd.DataFrame.from_dict(tier1_dic,orient='index',columns=['tier1'])
linear_data['tier2']=pd.DataFrame.from_dict(tier2_dic,orient='index',columns=['tier2'])
linear_data['tier3']=pd.DataFrame.from_dict(tier3_dic,orient='index',columns=['tier3'])
linear_data['sum'] = linear_data['tier1'] + linear_data['tier2'] + linear_data['tier3']
linear_data['avg_cac'] = 6000 / linear_data['sum']
linear_data=linear_data.sort_values(by='tier1',ascending=False)

linear_data

,tier1,tier2,tier3,sum,avg_cac
referral,1517.100000,2656.683333,3290.050000,7463.833333,0.803876
social,484.583333,822.366667,995.233333,2302.183333,2.606222
display,439.266667,724.083333,846.116667,2009.466667,2.985867
email,251.133333,442.516667,581.450000,1275.100000,4.705513
paid_search,128.050000,237.816667,339.000000,704.866667,8.512248
organic_search,1.950000,6.466667,7.733333,16.150000,371.517028
direct,1.916667,2.066667,3.416667,7.400000,810.810811


In [17]:
channel_spend

,tier,channel_spend
0,tier1,"{'email': 1000.0, 'social': 1000.0, 'display':..."
1,tier2,"{'email': 2000.0, 'social': 2000.0, 'display':..."
2,tier3,"{'email': 3000.0, 'social': 3000.0, 'display':..."


In [18]:
channel_data=pd.DataFrame(columns=linear_data.columns,index=linear_data.index)
for i in range(1,4):
    dic=eval(channel_spend['channel_spend'][i-1])
    channel_data['tier'+str(i)]=pd.DataFrame.from_dict(dic,orient='index',columns=['tier'+str(i)])
channel_data/linear_data

,tier1,tier2,tier3
referral,0.659152,0.752818,0.911840
social,2.063629,2.432005,3.014368
display,2.276521,2.762113,3.545610
email,3.981949,4.519604,5.159515
paid_search,7.809449,8.409840,8.849558
organic_search,0.000000,0.000000,0.000000
direct,0.000000,0.000000,0.000000


In [19]:
channel_data/first

,sum,tier1,tier2,tier3
referral,NaN,0.658328,0.735565,0.921093
social,NaN,2.105263,2.430134,3.054990
display,NaN,2.298851,2.754821,3.448276
email,NaN,3.787879,4.975124,4.854369
paid_search,NaN,7.751938,9.478673,9.146341
organic_search,NaN,0.000000,0.000000,0.000000
direct,NaN,0.000000,0.000000,0.000000


In [20]:
channel_data/last

,tier1,tier2,tier3
referral,0.825764,0.949217,1.112760
social,1.600000,1.886792,2.352941
display,1.893939,2.444988,3.171247
email,3.558719,3.603604,4.580153
paid_search,5.681818,5.763689,6.237006
organic_search,0.000000,0.000000,0.000000
direct,0.000000,0.000000,0.000000


#### Part 2

In [34]:
linear_data 

In [35]:
linear_data["marginal_2"] = linear_data["tier2"] - linear_data["tier1"]
linear_data["marginal_3"] = linear_data["tier3"] - linear_data["tier2"]
linear_data


,tier1,tier2,tier3,sum,avg_cac,marginal_2,marginal_3
referral,1519,2719,3257,7495,0.800534,1200,538
social,475,823,982,2280,2.631579,348,159
display,435,726,870,2031,2.954210,291,144
email,264,402,618,1284,4.672897,138,216
paid_search,129,211,328,668,8.982036,82,117
organic_search,1,7,7,15,400.000000,6,0
direct,1,4,1,6,1000.000000,3,-3


In [36]:
linear_data["Marginal_spend"] = np.array([1000,1000,1000,1000,1000,0,0])

In [37]:
linear_data["CAC_marginal_1"] = linear_data["Marginal_spend"] / linear_data["tier1"]
linear_data["CAC_marginal_2"] = linear_data["Marginal_spend"] / linear_data["marginal_2"]
linear_data["CAC_marginal_3"] = linear_data["Marginal_spend"] / linear_data["marginal_3"]
linear_data

,tier1,tier2,tier3,sum,avg_cac,marginal_2,marginal_3,Marginal_spend,CAC_marginal_1,CAC_marginal_2,CAC_marginal_3
referral,1519,2719,3257,7495,0.800534,1200,538,1000,0.658328,0.833333,1.858736
social,475,823,982,2280,2.631579,348,159,1000,2.105263,2.873563,6.289308
display,435,726,870,2031,2.954210,291,144,1000,2.298851,3.436426,6.944444
email,264,402,618,1284,4.672897,138,216,1000,3.787879,7.246377,4.629630
paid_search,129,211,328,668,8.982036,82,117,1000,7.751938,12.195122,8.547009
organic_search,1,7,7,15,400.000000,6,0,0,0.000000,0.000000,NaN
direct,1,4,1,6,1000.000000,3,-3,0,0.000000,0.000000,-0.000000


In [38]:
CAC_marginal = linear_data[["CAC_marginal_1", "CAC_marginal_2", "CAC_marginal_3"]]
CAC_marginal.drop(["direct", "organic_search"], inplace=True)
CAC_marginal

/var/folders/xc/wrkdv3595mq6x6b_yvs2v83c0000gn/T/ipykernel_28669/3896302397.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CAC_marginal.drop(["direct", "organic_search"], inplace=True)


,CAC_marginal_1,CAC_marginal_2,CAC_marginal_3
referral,0.658328,0.833333,1.858736
social,2.105263,2.873563,6.289308
display,2.298851,3.436426,6.944444
email,3.787879,7.246377,4.629630
paid_search,7.751938,12.195122,8.547009


#### find the optimal choice

In [39]:
optimal_channel = list(CAC_marginal.index)
optimal_spend = list(CAC_marginal.columns)
l = list()
for i in optimal_channel:
    for j in optimal_spend:
        l.append((i+' '+j, CAC_marginal.loc[i][j]))
l.sort(key=lambda x: x[1])
l[:11]

[('referral CAC_marginal_1', 0.6583278472679395),
 ('referral CAC_marginal_2', 0.8333333333333334),
 ('referral CAC_marginal_3', 1.858736059479554),
 ('social CAC_marginal_1', 2.1052631578947367),
 ('display CAC_marginal_1', 2.2988505747126435),
 ('social CAC_marginal_2', 2.8735632183908044),
 ('display CAC_marginal_2', 3.4364261168384878),
 ('email CAC_marginal_1', 3.787878787878788),
 ('email CAC_marginal_3', 4.62962962962963),
 ('social CAC_marginal_3', 6.289308176100629),
 ('display CAC_marginal_3', 6.944444444444445)]

In [40]:
dic = dict()
for i in optimal_channel:
    dic[i] = 0
for i in l[:11]:
    s = i[0].split()
    dic[s[0]] += 1000
dic

{'referral': 3000,
 'social': 3000,
 'display': 3000,
 'email': 2000,
 'paid_search': 0}